# Biodata
* Nama: **Virgiawan Teguh Kusuma**
* Email: **181240000833@unisnu.ac.id**
* Domisili: **Kabupaten Jepara, Jawa Tengah**

# Load Library

## Load google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Library lainnya

In [2]:
import pandas as pd
import zipfile, os

# Load Dataset



### Load dataset & Ekstraksi file yang masih berbentuk zip dari GDrive

In [3]:
phone = '/content/drive/MyDrive/archive_7.zip'
phoneRef = zipfile.ZipFile(phone, 'r')
phoneRef.extractall('/content')
phoneRef.close()

### Inisialisasi variable untuk membuat Dataframe

In [4]:
dataframe = pd.read_csv('mobiles1.csv')
dataframe

,battery,camera,display,memory,name,price,processor,rating,reviews,warranty
0,5000 mAh Battery,12MP + 2MP | 8MP Front Camera,15.8 cm (6.22 inch) HD+ Display,4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,"Redmi 8 (Ruby Red, 64 GB)",9999,Qualcomm Snapdragon 439 Processor,4.4,"55,078 Reviews",Brand Warranty of 1 Year Available for Mobile ...
1,5000 mAh Battery,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,4 GB RAM | 64 GB ROM,"Realme 5i (Aqua Blue, 64 GB)",10999,Qualcomm Snapdragon 665 2 GHz Processor,4.5,"20,062 Reviews",Sunrise Design
2,5000 mAh Battery,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,4 GB RAM | 128 GB ROM,"Realme 5i (Aqua Blue, 128 GB)",11999,Qualcomm Snapdragon 665 (2 GHz) Processor,4.5,"20,062 Reviews",Sunrise Design
3,5000 mAh Battery,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,4 GB RAM | 128 GB ROM,"Realme 5i (Forest Green, 128 GB)",11999,Qualcomm Snapdragon 665 (2 GHz) Processor,4.5,"20,062 Reviews",Sunrise Design
4,4000 mAh Battery,13MP + 2MP | 5MP Front Camera,15.49 cm (6.1 inch) HD+ Display,3 GB RAM | 32 GB ROM | Expandable Upto 256 GB,"Realme C2 (Diamond Blue, 32 GB)",7499,MediaTek P22 Octa Core 2.0 GHz Processor,4.4,"10,091 Reviews",Dual Nano SIM slots and Memory Card Slot
...,...,...,...,...,...,...,...,...,...,...
979,2000 mAh Battery,5MP Rear Camera | 2MP Front Camera,12.7 cm (5 inch) FWVGA Display,1 GB RAM | 8 GB ROM | Expandable Upto 32 MB,"Micromax Bharat 4 (Black, 8 GB)",3590,12 Months Brand Warranty,3.8,105 Reviews,NaN
980,2680 mAh Li-Ion Battery,13MP Rear Camera | 5MP Front Camera,13.21 cm (5.2 inch) Full HD Display,3 GB RAM | 32 GB ROM,"Nextbit Robin (Ember, 32 GB)",19999,Qualcomm Snapdragon 808 MSM8992 Processor,4.0,516 Reviews,Brand Warranty of 1 Year
981,4550 mAh Battery,13MP + 5MP | 20MP Front Camera,15.24 cm (6 inch) Full HD Display,4 GB RAM | 64 GB ROM | Expandable Upto 256 GB,"Gionee A1 Plus (Mocha Gold, 64 GB)",10499,Helio P25 MT 6757CD Processor,4.1,710 Reviews,Brand Warranty of 1 Year Available for Mobile ...
982,2100 mAh Li-Ion Battery,8MP Rear Camera | 2MP Front Camera,12.7 cm (5 inch) HD Display,1 GB RAM | 8 GB ROM | Expandable Upto 32 GB,"XOLO Omega 5.0 (Black, 8 GB)",8990,MTK 6592M Processor,3.8,81 Reviews,1 Year Manufacturer Warranty


# Data Cleaning

## Drop Kolom yang tidak digunakan

In [5]:
dataframe = dataframe.drop(columns=['camera',
                                    'display',
                                    'rating',
                                    'reviews',
                                    'warranty'])
dataframe.head()

,battery,memory,name,price,processor
0,5000 mAh Battery,4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,"Redmi 8 (Ruby Red, 64 GB)",9999,Qualcomm Snapdragon 439 Processor
1,5000 mAh Battery,4 GB RAM | 64 GB ROM,"Realme 5i (Aqua Blue, 64 GB)",10999,Qualcomm Snapdragon 665 2 GHz Processor
2,5000 mAh Battery,4 GB RAM | 128 GB ROM,"Realme 5i (Aqua Blue, 128 GB)",11999,Qualcomm Snapdragon 665 (2 GHz) Processor
3,5000 mAh Battery,4 GB RAM | 128 GB ROM,"Realme 5i (Forest Green, 128 GB)",11999,Qualcomm Snapdragon 665 (2 GHz) Processor
4,4000 mAh Battery,3 GB RAM | 32 GB ROM | Expandable Upto 256 GB,"Realme C2 (Diamond Blue, 32 GB)",7499,MediaTek P22 Octa Core 2.0 GHz Processor


## Melihat tipe data tiap kolom

In [6]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   battery    984 non-null    object
 1   memory     984 non-null    object
 2   name       984 non-null    object
 3   price      984 non-null    int64 
 4   processor  983 non-null    object
dtypes: int64(1), object(4)
memory usage: 38.6+ KB


## Split kolom memory dan display
* memisahkan **RAM** dengan ROM pada kolom memory menjadi masing-masing kolom baru.


In [7]:
split_memory = dataframe['memory'].str.split('|', 1, expand=True)

dataframe['RAM(GB)'] = split_memory[0]
dataframe['ROM(GB)'] = split_memory[1]
dataframe.head()

,battery,memory,name,price,processor,RAM(GB),ROM(GB)
0,5000 mAh Battery,4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,"Redmi 8 (Ruby Red, 64 GB)",9999,Qualcomm Snapdragon 439 Processor,4 GB RAM,64 GB ROM | Expandable Upto 512 GB
1,5000 mAh Battery,4 GB RAM | 64 GB ROM,"Realme 5i (Aqua Blue, 64 GB)",10999,Qualcomm Snapdragon 665 2 GHz Processor,4 GB RAM,64 GB ROM
2,5000 mAh Battery,4 GB RAM | 128 GB ROM,"Realme 5i (Aqua Blue, 128 GB)",11999,Qualcomm Snapdragon 665 (2 GHz) Processor,4 GB RAM,128 GB ROM
3,5000 mAh Battery,4 GB RAM | 128 GB ROM,"Realme 5i (Forest Green, 128 GB)",11999,Qualcomm Snapdragon 665 (2 GHz) Processor,4 GB RAM,128 GB ROM
4,4000 mAh Battery,3 GB RAM | 32 GB ROM | Expandable Upto 256 GB,"Realme C2 (Diamond Blue, 32 GB)",7499,MediaTek P22 Octa Core 2.0 GHz Processor,3 GB RAM,32 GB ROM | Expandable Upto 256 GB


## Mengubah tipe data
* Mengubah tipe data pada kolom **battery** menjadi *float* dan mengubah judul kolomnya menjadi **battery(mAh)**.
* Mengubah tipe data pada kolom **RAM** dan **ROM** menjadi *integer*.
* Mengubah tipe data pada kolom **price** menjadi *float* dan mengonversinya dari *Rupees* ke *IDR(Rupiah)* dengan mengalikan nilai 1 Rupees dalam IDR.

In [8]:
data_battery= dataframe['battery'].str.split('(\d+)',  expand=True)
data_ram= dataframe['RAM(GB)'].str.split('(\d+)',  expand=True)
data_rom= dataframe['ROM(GB)'].str.split('(\d+)',  expand=True)

dataframe['battery(mAh)'] = pd.to_numeric(data_battery[1], downcast='float')
dataframe['RAM(GB)'] = pd.to_numeric(data_ram[1], downcast='integer')
dataframe['ROM(GB)'] = pd.to_numeric(data_rom[1], downcast='integer')
dataframe['price(IDR)'] = (dataframe['price'] * 191.31 )

## Drop Kolom yang sudah tidak digunakan

In [9]:
dataframe = dataframe.drop(columns=['battery',
                                    'price', 
                                    'name',
                                    'memory',
                                    'processor'])
dataframe

,RAM(GB),ROM(GB),battery(mAh),price(IDR)
0,4,64,5000.0,1912908.69
1,4,64,5000.0,2104218.69
2,4,128,5000.0,2295528.69
3,4,128,5000.0,2295528.69
4,3,32,4000.0,1434633.69
...,...,...,...,...
979,1,8,2000.0,686802.90
980,3,32,2680.0,3826008.69
981,4,64,4550.0,2008563.69
982,1,8,2100.0,1719876.90


## Cek tipe data

In [10]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   RAM(GB)       984 non-null    int16  
 1   ROM(GB)       984 non-null    int16  
 2   battery(mAh)  984 non-null    float32
 3   price(IDR)    984 non-null    float64
dtypes: float32(1), float64(1), int16(2)
memory usage: 15.5 KB


## Filtering Smartphone
Menetapkan syarat minimal spesifikasi handphone di tahun 2021, yaitu:
* Minimal kapasitas baterai 3000 mAh
* RAM 3GB
* ROM/Penyimpanan Internal 32GB

In [11]:
loc_data=dataframe.loc[
                       (dataframe['battery(mAh)'] >= 3000) &
                       (dataframe['RAM(GB)'] >= 3) &
                       (dataframe['ROM(GB)'] >= 32)
                       ]

## Inisialisai variable baru hasil Filtering

In [12]:
newDataframe = loc_data

## Cek Data

In [13]:
newDataframe.head()

,RAM(GB),ROM(GB),battery(mAh),price(IDR)
0,4,64,5000.0,1912908.69
1,4,64,5000.0,2104218.69
2,4,128,5000.0,2295528.69
3,4,128,5000.0,2295528.69
4,3,32,4000.0,1434633.69


## Cek Tipe Data

In [14]:
newDataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 731 entries, 0 to 983
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   RAM(GB)       731 non-null    int16  
 1   ROM(GB)       731 non-null    int16  
 2   battery(mAh)  731 non-null    float32
 3   price(IDR)    731 non-null    float64
dtypes: float32(1), float64(1), int16(2)
memory usage: 17.1 KB


## Cek data ada berapa baris dan kolom

In [15]:
newDataframe.shape

(731, 4)

## Cek statistik description dari data (mean, kuartil, standard deviation, dll.)

In [16]:
newDataframe.describe()

,RAM(GB),ROM(GB),battery(mAh),price(IDR)
count,731.000000,731.000000,731.000000,7.310000e+02
mean,5.149111,95.124487,4144.493652,3.567210e+06
std,2.003677,66.297954,670.548889,2.556974e+06
min,3.000000,32.000000,3000.000000,9.546369e+05
25%,4.000000,64.000000,3782.500000,2.007703e+06
50%,4.000000,64.000000,4000.000000,2.869459e+06
75%,6.000000,128.000000,4500.000000,4.017319e+06
max,12.000000,512.000000,6000.000000,2.008736e+07


## Cek missing value

In [17]:
newDataframe.isnull().sum()

RAM(GB)         0
ROM(GB)         0
battery(mAh)    0
price(IDR)      0
dtype: int64

In [18]:
newDataframe

,RAM(GB),ROM(GB),battery(mAh),price(IDR)
0,4,64,5000.0,1912908.69
1,4,64,5000.0,2104218.69
2,4,128,5000.0,2295528.69
3,4,128,5000.0,2295528.69
4,3,32,4000.0,1434633.69
...,...,...,...,...
974,6,64,4000.0,4015596.90
975,6,64,4000.0,4780836.90
976,6,64,5000.0,2678148.69
981,4,64,4550.0,2008563.69


# Data Modeling

## Inisialisasi variabel x dan y untuk nanti split data

In [19]:
x = newDataframe.drop(columns='price(IDR)')
y = newDataframe['price(IDR)']

## Split data menjadi train & test dengan perbandingan 80:20.


### Import library train_test_split

In [20]:
from sklearn.model_selection import train_test_split

### Split data

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=4)

# Linear Regression


In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, accuracy_score, mean_squared_error

In [28]:
lreg = LinearRegression()

In [29]:
lreg.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [30]:
lreg_predict = lreg.predict(x_test)

lreg_df = pd.DataFrame({'y_true': y_test,
                          'y_predict': lreg_predict})

lreg_mse = mean_squared_error(y_test, lreg_predict)
lreg_mae = mean_absolute_error(y_test, lreg_predict)

lreg_df.head()

,y_true,y_predict
295,9565308.69,9865286.00
45,3347733.69,4406192.00
36,2102496.90,1581327.00
901,2869458.69,2634123.25
224,4399938.69,7155364.00


## Mencari nilai koefisien dan intercept.

In [31]:
print(lreg.coef_)
print(lreg.intercept_)

[ 6.953526e+05  9.682177e+03 -2.382959e+02]
67089.25


## Cek accuracy score dari model yang sudah di train

In [32]:
lreg.score(x_test, y_test)

0.5900831226897991

# Random Forest Regression

In [34]:
from sklearn.ensemble import RandomForestRegressor

In [35]:
rf_model = RandomForestRegressor(n_estimators=50, max_depth=16,
                                 random_state=55, n_jobs=-1)
rf_model.fit(x_train, y_train)
rf_predict = rf_model.predict(x_test)

rf_df = pd.DataFrame({'y_true': y_test,
                          'y_predict': rf_predict})

rf_mse = mean_squared_error(y_test, rf_predict)
rf_mae = mean_absolute_error(y_test, rf_predict)
rf_df.head()

,y_true,y_predict
295,9565308.69,1.087097e+07
45,3347733.69,4.351822e+06
36,2102496.90,2.154782e+06
901,2869458.69,2.549039e+06
224,4399938.69,5.711560e+06


# Gradient Booster

In [36]:
from xgboost import XGBRegressor

In [37]:
xgb_model = XGBRegressor()
xgb_model.fit(x_train, y_train)
xgb_predict = xgb_model.predict(x_test)

xgb_df = pd.DataFrame({'y_true': y_test,
                          'y_predict': xgb_predict})

xgb_mse = mean_squared_error(y_test, xgb_predict)
xgb_mae = mean_absolute_error(y_test, xgb_predict)

xgb_df.head()

[14:05:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


,y_true,y_predict
295,9565308.69,9047948.00
45,3347733.69,4283885.00
36,2102496.90,2165378.50
901,2869458.69,2455154.75
224,4399938.69,5935549.00


In [38]:
metrics_df = pd.DataFrame({'MSE': [lreg_mse, rf_mse, xgb_mse],
                           'MAE': [lreg_mae, rf_mae, xgb_mae]},
                          index=['Linear Regression', 'Random Forest', 'Gradient Booster'])
metrics_df

,MSE,MAE
Linear Regression,2.030860e+12,955508.772891
Random Forest,2.422468e+12,844544.065076
Gradient Booster,2.514010e+12,896406.491429


# Mulai memprediksi harga smartphone berdasarkan spesifikasi:
<!-- * nilai pertama adalah **RAM**
* nilai kedua adalah **ROM/Peyimpanan Internal**
* nilai ketiga adalah **battery** -->

In [39]:
ram = int(input('masukkan kapasitas RAM yang diinginkan (tulis angkanya saja) : '))

masukkan kapasitas RAM yang diinginkan (tulis angkanya saja) : 4


In [40]:
rom = int(input('masukkan kapasitas ROM yang diinginkan (tulis angkanya saja) : '))

masukkan kapasitas ROM yang diinginkan (tulis angkanya saja) : 64


In [41]:
battery = float(input('masukkan kapasitas Baterai yang diinginkan (tulis angkanya saja) : '))

masukkan kapasitas Baterai yang diinginkan (tulis angkanya saja) : 5000


In [42]:
print(type(ram))
print(type(rom))
print(type(battery))

<class 'int'>
<class 'int'>
<class 'float'>


In [43]:
price = lreg.predict([[ram,rom,battery]])

## Import Library numpy

In [44]:
import numpy as np

## Konversi hasil prediksi dari array ke desimal

In [45]:
price = np.round([price], decimals=1)

## Print perkiraan harga smartphone sesuai spesifikasi yang diinginkan

In [46]:
print('Perkiraan Harga Smartphone yang Anda inginkan adalah Rp.',price[0][0])

Perkiraan Harga Smartphone yang Anda inginkan adalah Rp. 2276679.6
